In [ ]:
import csv
import json
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import re

In [ ]:
#Function to translate the csv file to json format and create a temporary .json file
#and create an initial DataFrame object
def data_read_inventory():
    with open ("Inventroy.csv", "r") as csv_file:
        reader = csv.DictReader(csv_file)
        data = []
        for row in reader:
            data.append(row)
    
    with open("temporary_Inventory_json.json","w") as json_file:
        json.dump(data, json_file, indent=4)
        
    df = pd.read_csv('Inventroy.csv')    
    return df

In [ ]:
#Function to translate the csv file to json format and create a temporary .json file
#and create an initial DataFrame object
def data_read_inspections():
    with open ("Inspections.csv", "r") as csv_file:
        reader = csv.DictReader(csv_file)
        data = []
        for row in reader:
            data.append(row)
    
    with open("temporary_Inspections_json.json","w") as json_file:
        json.dump(data, json_file, indent=4)
    
    df = pd.read_csv('Inspections.csv')   
    return df

In [ ]:
#Function to translate the csv file to json format and create a temporary .json file
#and create an initial DataFrame object
def data_read_violations():
    with open ("violations.csv", "r") as csv_file:
        reader = csv.DictReader(csv_file)
        data = []
        for row in reader:
            data.append(row)
    
    with open("temporary_Violations_json.json","w") as json_file:
        json.dump(data, json_file, indent=4)
    
    df = pd.read_csv('violations.csv')    
    return df

In [ ]:
def data_clean_inventory(df):
    
    #Column names cleaning
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')\
.str.replace('(', '').str.replace(')', '')
    
    df.rename(columns={'facility__state': 'facility_state'}, inplace=True)
    
    #Creating new column named "seating_number_and_type"
    df.insert(6, "seating_number_type", True)
    
    #Using a regex to extract the information to the new column
    df['seating_number_type'] = df.pe_description.str.extract(r'(\([^)]*\))',  
                                     expand = True)
    df['pe_description'] = df.pe_description.str.replace(r'(\([^)]*\))', '')
    df['pe_description'] = df.pe_description.str.replace(r'  ', ' ')
    
    #Cleaning the states columns (facility_state , owner_state)
    df['facility_state'] = df.facility_state.str.replace(r'`', 'CA')
    df['owner_state'] = df.owner_state.str.replace(r'`', 'NO STATE GIVEN')
    
    #Replace all multiple spaces with single
    df = df.replace(['      '], ' ', regex = True)
    df = df.replace(['     '], ' ', regex = True)
    df = df.replace(['    '], ' ', regex = True)
    df = df.replace(['   '], ' ', regex = True)
    df = df.replace(['  '], ' ', regex = True)
    
    #Replace unicode or special characters
    df = df.replace(["&#160;"]," ", regex = True)
    df = df.replace(["‚Äô"],"'", regex = True)
    df = df.replace(["√ë"],"Ñ", regex = True) 
    df = df.replace(["‚Ä∫"],"", regex = True)
    df = df.replace(["√â"],"É", regex = True)
    df = df.replace(["¬¢"],"", regex = True)
    df = df.replace(["85*C"],"85C", regex = True)
    df = df.replace(["85 C"],"85C", regex = True)
    df = df.replace(["¬∞ "],"", regex = True)
    
    df = df.replace(["CAPRIOTI"],"CAPRIOTTI", regex = True)
    
    #Replace semicolon with apostrophe
    df = df.replace([';'], "'", regex = True)
    
    #Drop the program_name column as it duplicates the data with facility_name column. 
    df.drop('program_name', inplace=True, axis=1)
    
    #Drop facility_latitude and facility_longitude as they duplicate information from location column
    df.drop('facility_latitude', inplace=True, axis=1)
    df.drop('facility_longitude', inplace=True, axis=1)        
    
    #Capitalise columns to keep consistent uppercase naming
    df['facility_address'] = df['facility_address'].str.upper()
    df['facility_city'] = df['facility_city'].str.upper()
    df['facility_state'] = df['facility_state'].str.upper()
    df['owner_name'] = df['owner_name'].str.upper()
    df['owner_address'] = df['owner_address'].str.upper()
    df['owner_city'] = df['owner_city'].str.upper()
    df['owner_state'] = df['owner_state'].str.upper()
    
    #Convert float values to integer in specific columns and fill missing values with 0
    
    df['census_tracts_2010'] = df['census_tracts_2010'].fillna(0.0).astype(np.int64)
    df['census_tracts_2010'] = df['census_tracts_2010'].astype(np.int64)
    
    df['2011_supervisorial_district_boundaries_official'] = df['2011_supervisorial\
_district_boundaries_official'].fillna(0.0).astype(np.int64)
    df['2011_supervisorial_district_boundaries_official'] = df['2011_supervisorial\
_district_boundaries_official'].astype(np.int64)
    
    df['board_approved_statistical_areas'] = df['board_approved_statistical_areas'].fillna(0.0).astype(np.int64)
    df['board_approved_statistical_areas'] = df['board_approved_statistical_areas'].astype(np.int64)

    df['zip_codes'] = df['zip_codes'].fillna(0.0).astype(np.int64)
    df['zip_codes'] = df['zip_codes'].astype(np.int64)
    
    #Filling NaN values 
    df['owner_city'] = df['owner_city'].fillna("NO CITY GIVEN")
    df['owner_state'] = df['owner_state'].fillna("NO STATE GIVEN")
    df['owner_zip'] = df['owner_zip'].fillna("NO ZIP CODE GIVEN")
    df['location'] = df['location'].fillna("NO LOCATION")
    
    return df

In [ ]:
def data_clean_inspections(df):
    #Column names cleaning
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')\
.str.replace('(', '').str.replace(')', '')
    
    #Creating new column named "seating_number_and_type"
    df.insert(11, "seating_number_type", True)
    
    #Using a regex to extract the information to the new column
    df['seating_number_type'] = df.pe_description.str.extract(r'(\([^)]*\))',  
                                     expand = True)
    df['pe_description'] = df.pe_description.str.replace(r'(\([^)]*\))', '')
    df['pe_description'] = df.pe_description.str.replace(r'  ', ' ')
    
    #Cleaning the facility_state column
    df['facility_state'] = df.facility_state.str.replace(r'`', 'CA')
    
    #Strip word "point" from location column
    df['location'] = df.location.str.replace(r'(POINT)', '', regex=True)
    
    #Replace all multiple spaces with single
    df = df.replace(['      '], ' ', regex = True)
    df = df.replace(['     '], ' ', regex = True)
    df = df.replace(['    '], ' ', regex = True)
    df = df.replace(['   '], ' ', regex = True)
    df = df.replace(['  '], ' ', regex = True)
    
    #Replace unicode or special characters
    df = df.replace(["&#160;"]," ", regex = True)
    df = df.replace(["‚Äô"],"'", regex = True)
    df = df.replace(["√ë"],"Ñ", regex = True) 
    df = df.replace(["‚Ä∫"],"", regex = True)
    df = df.replace(["√â"],"É", regex = True)
    df = df.replace(["¬¢"],"", regex = True)
    df = df.replace(["85*C"],"85C", regex = True)
    df = df.replace(["85 C"],"85C", regex = True)
    df = df.replace(["¬∞ "],"", regex = True)
    
    df = df.replace(["CAPRIOTI"],"CAPRIOTTI", regex = True)
    
    #Replace semicolon with apostrophe
    df = df.replace([';'], "'", regex = True)
    
    #Drop the program_name column as it duplicates the data with facility_name column    
    df.drop('program_name', inplace=True, axis=1)
    
    #Capitalise columns to keep consistent uppercase naming
    df['owner_name'] = df['owner_name'].str.upper()
    df['facility_name'] = df['facility_name'].str.upper()
    df['facility_state'] = df['facility_state'].str.upper()
    df['facility_address'] = df['facility_address'].str.upper()
    df['facility_city'] = df['facility_city'].str.upper()
    df['facility_state'] = df['facility_state'].str.upper()
    
    #Convert float values to integer in specific columns and fill missing values with 0 
    df['score'] = df['score'].fillna(0.0).astype(np.int64)
    df['score'] = df['score'].astype(np.int64)
    
    df['census_tracts_2010'] = df['census_tracts_2010'].fillna(0.0).astype(np.int64)
    df['census_tracts_2010'] = df['census_tracts_2010'].astype(np.int64)
    
    df['2011_supervisorial_district_boundaries_official'] = df['2011_supervisorial\
_district_boundaries_official'].fillna(0.0).astype(np.int64)
    df['2011_supervisorial_district_boundaries_official'] = df['2011_supervisorial\
_district_boundaries_official'].astype(np.int64)
    
    df['board_approved_statistical_areas'] = df['board_approved_statistical_areas'].fillna(0.0).astype(np.int64)
    df['board_approved_statistical_areas'] = df['board_approved_statistical_areas'].astype(np.int64)

    df['zip_codes'] = df['zip_codes'].fillna(0.0).astype(np.int64)
    df['zip_codes'] = df['zip_codes'].astype(np.int64)
    
    #Filling NaN values 
    df['seating_number_type'] = df['seating_number_type'].fillna("PRIVATE")
    df['grade'] = df['grade'].fillna("NO GRADE GIVEN")
    df['location'] = df['location'].fillna("NO LOCATION GIVEN")
    
    #Replacing the seating_number_type column values to more descriptive names
    replace_values = {'(0-10)':'(0-10 SEATS)', '(0-1999 SF)':'(0-1999 SQUARE FEET)', '(0-30)':'(0-30 SEATS)',
                  '(0-999 SQ. FT.)':'(0-999 SQUARE FEET)', '(1,000-1,999 SQ. FT.)':'(1,000-1,999 SQUARE FEET)',
                  '(1-1,999 SF)':'(1-1,999 SQUARE FEET)','(1-1,999 SQ. FT.)':'(1-1,999 SQUARE FEET)',
                  '(1-1,999)':'(1-1,999 SEATS)', '(1-4,999)':'(1-4,999 SEATS)', '(10,000+ SF)':'(10,000+ SQUARE FEET)',
                  '(151 + )':'(151+ SEATS)', '(2,000+ SF)':'(2,000+ SQUARE FEET)', '(2,000-4,999)':'(2,000-4,999 SEATS)',
                  '(2,000-5,999 SF)':'(2,000-5,999 SQUARE FEET)', '(2000-3999 SF)':'(2000-3999 SQUARE FEET)',
                  '(31-60)':'(31-60 SEATS)', '(4000-9999 SF)':'(4000-9999 SQUARE FEET)',
                  '(5,000 + )':'(5,000+ SEATS)', '(61-150)':'(61-150 SEATS)'}
    
    df = df.replace({"seating_number_type": replace_values})
    
    return df

In [ ]:
def data_clean_violations(df):
    #Column names cleaning
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')\
.str.replace('(', '').str.replace(')', '')
    
    df.rename(columns={'violation__status': 'violation_status'}, inplace=True)
    
    #Remove duplicates from violation_code and violation_description columns
    
    #Strip numerical values out of violation_description column so the violation_code 
    #corresponds to each violation_description value
    pattern = '[0-9]'
    df['violation_description'] = df['violation_description'].str.replace(pattern,'')
    
    df['violation_description'] = df['violation_description'].str.replace('# . ','')
    df['violation_description'] = df['violation_description'].str.replace('# a. ','')
    df['violation_description'] = df['violation_description'].str.replace('# b. ','')
    df['violation_description'] = df['violation_description'].str.replace('.','')
    df['violation_description'] = df['violation_description'].str.replace('  ',' ')
    
    df['violation_description'] = df['violation_description'].str.replace('Equipment/utensils \
approved;','Equipment/Utensils - approved;')
    
    #Remove duplicate violation_code values so each violation_code corresponds to each violation_description value   
    replace_values = {'MF08':'F007', 'MF15':'F014', 'MF31':'F030', 'MF34':'F033', 'MF36':'F035', 'MF38':'F037',
                     'MF45':'F044', 'F055':'F001', 'F056':'F010', 'F057':'F018', 'F058':'F019', 'W001':'F007', 'W048':'F048'}
    
    df = df.replace({"violation_code": replace_values})
    
    return df